In [5]:
from functions import *
from functions import evaluate_dax
from functions import get_DAX
from models.DAX import DAX_baseline
from models.DAX import DAX_quantile_regression

In [6]:
daxdata=get_DAX.get()

baseline_model = {
    'name': 'Baseline Model',
    'function': DAX_baseline.DAX_baseline
}
quantile_model = {
    'name': 'Quantile Regression',
    'function': DAX_quantile_regression.DAX_quantile_regression
}



In [7]:
evaluation_df=evaluate_dax.evaluate_daxmodel(baseline_model,quantile_model,daxdata,last_x=5,weeks=True)

Evaluating models: 100%|██████████| 3/3 [00:11<00:00,  3.95s/it]


In [8]:
evaluation_df

(Empty DataFrame
 Columns: [forecast_date, target, horizon, q0.025, q0.25, q0.5, q0.75, q0.975, ret1]
 Index: [],
 Empty DataFrame
 Columns: [forecast_date, target, horizon, q0.025, q0.25, q0.5, q0.75, q0.975, ret1]
 Index: [])

In [5]:
df=DAX_baseline.DAX_baseline(daxdata)

In [6]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-18,DAX,1 day,-3.321265,-0.551383,0.060969,0.720873,2.719772
1,2023-11-18,DAX,2 day,-4.087895,-0.823250,0.072355,0.987420,3.570492
2,2023-11-18,DAX,5 day,-5.259161,-0.917655,0.161798,1.224285,4.628697
3,2023-11-18,DAX,6 day,-6.282629,-1.044070,0.186070,1.440815,5.367396
4,2023-11-18,DAX,7 day,-6.735742,-1.091646,0.326190,1.573729,6.259934


In [3]:
import pandas as pd
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm

def evaluate_daxmodel(model1, model2, df, last_x=100, years=False, months=False, weeks=False):
    df_before = df
    evaluation_model1 = pd.DataFrame()
    evaluation_model2 = pd.DataFrame()

    for w in tqdm(range(2, last_x), desc="Evaluating models"):
        df_before, df_after = split_time(df_before, num_years=years, num_months=months, num_weeks=weeks)

        # Evaluate model1
        pred_model1 = model1['function'](df_before)
        evaluation_model1 = evaluate_and_append(evaluation_model1, pred_model1, df, w)

        # Evaluate model2
        pred_model2 = model2['function'](df_before)
        evaluation_model2 = evaluate_and_append(evaluation_model2, pred_model2, df, w)

    return evaluation_model1, evaluation_model2

def evaluate_and_append(evaluation_df, pred, df, horizon):
    # Match predictions with observed values based on forecast_date and horizon
    pred['forecast_date'] = pd.to_datetime(pred['forecast_date'])
    df['Date'] = pd.to_datetime(df['Date'])

    # For each prediction, find the corresponding observed value
    for index, row in pred.iterrows():
        forecast_date = row['forecast_date']
        target_horizon = horizon
        target_date = forecast_date + pd.DateOffset(days=target_horizon)

        if target_date in df['Date'].values:
            observed_value = df.loc[df['Date'] == target_date, 'ret1'].iloc[0]
            pred.at[index, 'observed_ret1'] = observed_value
        else:
            pred.at[index, 'observed_ret1'] = None  # Or some placeholder value

    # Calculate score for each prediction
    pred['score'] = pred.apply(lambda row: evaluate_horizon(
        row[['q0.025', 'q0.25', 'q0.5', 'q0.75', 'q0.975']], row['observed_ret1']), axis=1)

    evaluation_df = pd.concat([evaluation_df, pred])
    return evaluation_df
evaluation_df=evaluate_dax.evaluate_daxmodel(baseline_model,quantile_model,daxdata,last_x=5,weeks=True)

Evaluating models: 100%|██████████| 3/3 [00:10<00:00,  3.42s/it]


In [4]:
evaluation_df

(Empty DataFrame
 Columns: [forecast_date, target, horizon, q0.025, q0.25, q0.5, q0.75, q0.975, ret1]
 Index: [],
 Empty DataFrame
 Columns: [forecast_date, target, horizon, q0.025, q0.25, q0.5, q0.75, q0.975, ret1]
 Index: [])